### Getting the outputs of the model
Author: Juraj Dedič

In [9]:
#import docbin
import spacy
from spacy.tokens import DocBin
from spacy import displacy
from spacy.scorer import Scorer
from spacy.training import Example
import spacy_transformers

nlp = spacy.load(R"results/s2/model-best") #load the best model
# test_sentences = [x[0] for x in TEST_DATA[0:100]] # extract the sentences from [sentence, entity]

In [11]:
db_test_correct = DocBin().from_disk(R"./v4-dataset/test2_spancat.spacy") #load the validation data
docs_test_correct = db_test_correct.get_docs(nlp.vocab) #get the validation data

scorer = Scorer()
examples = []

for doc_correct in docs_test_correct:
    text = doc_correct.text
    doc_predict = nlp(text) # predict the entities

    #create the example
    example = Example(doc_predict, doc_correct)
    examples.append(example)

    displacy.render(doc_predict, style="span", jupyter=True) #visualize the predicted entities
    displacy.render(doc_correct, style="span", jupyter=True) #visualize the correct entities

    spans = doc_predict.spans["sc"]
    for span, confidence in zip(spans, spans.attrs["scores"]):
        if confidence > 0.0:
            print(span.label_, span.start, span.end, confidence)
            
    break

res = scorer.score(examples)

CALLSIGN 0 3 0.00015218343
CALLSIGN 1 4 1.8532863e-05
CALLSIGN 2 5 0.001185202
CALLSIGN 3 6 2.5894728e-06
CALLSIGN 4 7 7.5396656e-06
CALLSIGN 5 8 1.7177968e-05
CALLSIGN 6 9 1.498354e-05
CALLSIGN 7 10 2.0582378e-05
CALLSIGN 8 11 1.2257475e-05
CALLSIGN 9 12 1.2507957e-05
CALLSIGN 10 13 1.981259e-05
CALLSIGN 11 14 1.6397742e-05
CALLSIGN 12 15 1.2081502e-05
CALLSIGN 13 16 1.9881316e-05
CALLSIGN 14 17 2.9077615e-05
CALLSIGN 15 18 1.8597202e-05
CALLSIGN 16 19 0.0003466967
CALLSIGN 17 20 7.2065213e-06
CALLSIGN 0 4 0.00047851435
CALLSIGN 1 5 0.00070448144
CALLSIGN 2 6 3.558627e-06
CALLSIGN 3 7 2.6513935e-06
CALLSIGN 4 8 1.5294627e-05
CALLSIGN 5 9 1.4422501e-05
CALLSIGN 6 10 1.6879188e-05
CALLSIGN 7 11 1.3649165e-05
CALLSIGN 8 12 1.588806e-05
CALLSIGN 9 13 1.4498097e-05
CALLSIGN 10 14 1.5961123e-05
CALLSIGN 11 15 1.3148771e-05
CALLSIGN 12 16 2.1479684e-05
CALLSIGN 13 17 2.1741309e-05
CALLSIGN 14 18 2.6883254e-05
CALLSIGN 15 19 0.0005606568
CALLSIGN 16 20 1.0865627e-05
CALLSIGN 0 5 0.9995121
CAL

In [14]:
from flask import Flask, request, jsonify

In [19]:
app = Flask(__name__)

@app.route("/", methods=['POST'])
def hello_world():
    content_type = request.headers.get('Content-Type')
    if (content_type == 'application/json'):
        json = request.json
        text = json['text']
        doc_ner = nlp(text)
        print(doc_ner.spans["sc"])
        spans = [span for span in doc_ner.spans["sc"] if span.label_ == "CALLSIGN"]
        scores = spans.attrs["scores"]
        #add the scores to the spans
        for span, score in zip(spans, scores):
            span._.set("score", score)
        #sort the spans by score
        spans = sorted(spans, key=lambda span: span._.score, reverse=True)
        print(spans)
        return jsonify([span.text for span in spans])
    else:
        return 'Content-Type not supported!'
    
app.run(port=6000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:6000
Press CTRL+C to quit


In [13]:
res

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'sents_p': None,
 'sents_r': None,
 'sents_f': None,
 'tag_acc': None,
 'pos_acc': None,
 'morph_acc': None,
 'morph_micro_p': None,
 'morph_micro_r': None,
 'morph_micro_f': None,
 'morph_per_feat': None,
 'dep_uas': None,
 'dep_las': None,
 'dep_las_per_type': None,
 'ents_p': None,
 'ents_r': None,
 'ents_f': None,
 'ents_per_type': None,
 'cats_score': 0.0,
 'cats_score_desc': 'macro F',
 'cats_micro_p': 0.0,
 'cats_micro_r': 0.0,
 'cats_micro_f': 0.0,
 'cats_macro_p': 0.0,
 'cats_macro_r': 0.0,
 'cats_macro_f': 0.0,
 'cats_macro_auc': 0.0,
 'cats_f_per_type': {},
 'cats_auc_per_type': {}}